In [66]:
pip install streamlit langchain-community langchain-openai langchain-text-splitters requests pathlib pypdf

Note: you may need to restart the kernel to use updated packages.


In [67]:
## Handle secrets either from env vars or streamlit manager
import streamlit as st
import os
api_key = os.getenv("LITELLM_KEY")
if api_key is None:
    api_key = st.secrets["LITELLM_KEY"]
cirrus_key = os.getenv("CIRRUS_KEY")
if cirrus_key is None:
    cirrus_key = st.secrets["CIRRUS_KEY"]        


In [68]:
"""
import os
import requests
import zipfile

def download_and_unzip(url, output_dir):
    response = requests.get(url)
    zip_file_path = os.path.basename(url)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    os.remove(zip_file_path)

download_and_unzip("https://minio.carlboettiger.info/public-data/hwc.zip", "hwc")
"""

'\nimport os\nimport requests\nimport zipfile\n\ndef download_and_unzip(url, output_dir):\n    response = requests.get(url)\n    zip_file_path = os.path.basename(url)\n    with open(zip_file_path, \'wb\') as f:\n        f.write(response.content)\n    with zipfile.ZipFile(zip_file_path, \'r\') as zip_ref:\n        zip_ref.extractall(output_dir)\n    os.remove(zip_file_path)\n\ndownload_and_unzip("https://minio.carlboettiger.info/public-data/hwc.zip", "hwc")\n'

In [69]:
import pathlib
from langchain_community.document_loaders import PyPDFLoader
"""
def pdf_loader(path):
    all_documents = []
    docs_dir = pathlib.Path(path)
    for file in docs_dir.iterdir():
        loader = PyPDFLoader(file)
        documents = loader.load()
        all_documents.extend(documents)
    return all_documents

docs = pdf_loader('hwc/')
docs
"""

"\ndef pdf_loader(path):\n    all_documents = []\n    docs_dir = pathlib.Path(path)\n    for file in docs_dir.iterdir():\n        loader = PyPDFLoader(file)\n        documents = loader.load()\n        all_documents.extend(documents)\n    return all_documents\n\ndocs = pdf_loader('hwc/')\ndocs\n"

In [70]:

# NRP embedding model tends to throw errors
# embedding = OpenAIEmbeddings(model = "embed-mistral", api_key = api_key, base_url = "https://llm.nrp-nautilus.io")



In [71]:
## Use the model on Cirrus instead:
"""
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
                 model = "cirrus",
                 api_key =  cirrus_key, 
                 base_url = "https://llm.cirrus.carlboettiger.info/v1",
)
"""

'\nfrom langchain_openai import OpenAIEmbeddings\nembedding = OpenAIEmbeddings(\n                 model = "cirrus",\n                 api_key =  cirrus_key, \n                 base_url = "https://llm.cirrus.carlboettiger.info/v1",\n)\n'

In [72]:
# Build a retrival agent
"""
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
splits = text_splitter.split_documents(docs)
"""

'\nfrom langchain_text_splitters import RecursiveCharacterTextSplitter\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)\nsplits = text_splitter.split_documents(docs)\n'

In [73]:
# slow part here, runs on remote GPU
"""
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore.from_documents(documents = splits, embedding = embedding)
retriever = vectorstore.as_retriever()
"""

'\nfrom langchain_core.vectorstores import InMemoryVectorStore\nvectorstore = InMemoryVectorStore.from_documents(documents = splits, embedding = embedding)\nretriever = vectorstore.as_retriever()\n'

In [74]:
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI

documents = [
    Document(
        page_content=(
            "Abstract\n"
            "African lions (Panthera leo) are in decline across many parts of the continent, with retaliatory killing for livestock attacks being a major cause. "
            "In East Africa, projects are fortifying bomas (livestock enclosures) to reduce conflict between large carnivores and people, aiming to prevent the indiscriminate killing of lions. "
            "However, the lack of evidence-based studies has made it difficult to scientifically verify the effectiveness of these interventions. "
            "This study evaluates fortified bomas—called Living Walls—by comparing carnivore attack rates at 84 unprotected bomas and 62 fortified ones. "
            "The Living Walls were 99.9% effective at preventing nighttime carnivore attacks over 1,790 boma-months, with no lion deaths occurring at fortified sites. "
            "These findings emphasize the importance of predator-proof enclosures for lion conservation in human–livestock-dominated landscapes.\n\n"
            "Keywords: African lion, Maasai, Large carnivores, Community-based conservation, Tanzania, East Africa\n\n"
            "Introduction\n"
            "Much of conservation literature emphasizes problems, with few studies rigorously testing the effectiveness of solutions. "
            "Lions are declining across Africa, especially outside protected areas, due to retaliatory killings following livestock predation. "
            "This mirrors challenges faced by other large carnivores globally. In response, East African conservation programs are promoting fortified bomas to reduce predation and associated retaliatory killings. "
            "Despite the growing adoption of such interventions, few empirical studies have assessed their real-world impact. "
            "This study seeks to answer: how effective are fortified bomas in reducing carnivore attacks and supporting lion conservation?\n\n"
            "Materials and Methods\n"
            "We monitored 146 permanent bomas on the Maasai Steppe near Tarangire National Park in northern Tanzania over a 10-year period, covering 9,296 boma-months. "
            "Depredation events were verified by trained Maasai community members, who recorded attack locations, times, and photographic evidence where possible. "
            "In total, 274 carnivore attacks were verified, with 68 attacks on the permanent bomas included in this study. "
            "These data provide a minimum estimate of conflict, acknowledging underreporting may have occurred."
        ),
        metadata={
            "title": "Evidence-based conservation: predator-proof bomas protect livestock and lions",
        }
    ),
    Document(
        page_content=(
            "Abstract\n"
            "The global lion (Panthera leo) population decline is partly a result of retaliatory killing in response to livestock depredation. "
            "Nairobi National Park (NNP) in Kenya, surrounded by human settlements, is one such area. "
            "To mitigate conflict, communities have implemented LED flashlight systems at livestock bomas to deter nocturnal lion attacks. "
            "We assessed lion responses to these flashlights from 2007 to 2016 by interviewing 80 boma owners—43 with flashlights and 37 without—and comparing 10 years of predation data. "
            "Bomas equipped with flashlights experienced significantly fewer attacks. "
            "After flashlight installation, predation events shifted away from fortified bomas and became more diurnal. "
            "These findings suggest the flashlight method is an effective tool for reducing nocturnal lion predation and improving coexistence.\n\n"
            "Introduction\n"
            "Lion populations are declining due to habitat loss, prey depletion, and retaliatory killings. "
            "These challenges are particularly acute where human-livestock densities are high. "
            "Across Kenya, Namibia, and Botswana, lions are frequently killed after attacking livestock. "
            "This is also a major concern in West and Central Africa, where populations are critically low. "
            "This study investigates a community-based, non-lethal deterrent method—LED flashlights—and its potential to reduce human-lion conflict near Nairobi National Park.\n\n"
            "Results show LED flashlights significantly reduce nighttime livestock predation and shift lion activity patterns. "
            "Broader implementation of this low-cost strategy could support lion conservation in similar settings.\n\n"
            "Data Availability: All relevant data are within the paper and its Supporting Information files.\n"
            "Funding: This study was part of the PhD project of the lead author, supported by the Louwes Fund for Water and Food, Leiden University, and the Leo Foundation."
        ),
        metadata={
            "title": "Effectiveness of a LED flashlight technique in reducing livestock depredation by lions (Panthera leo) around Nairobi National Park",
        }
    )
]


In [75]:
models = {"llama3": "llama3-sdsc", 
          "deepseek-small": "DeepSeek-R1-Distill-Qwen-32B",
          "deepseek": "deepseek-r1-qwen-qualcomm",
          "gemma3": "gemma3",
          "phi3": "phi3",
          "olmo": "olmo"
         }

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = models['gemma3'],
                 api_key = api_key, 
                 base_url = "https://llm.nrp-nautilus.io",  
                 temperature=0)

"""
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following scientific articles as the retrieved context to answer "
    "the question. Appropriately cite the articles from the context on which your answer is based using (Author, Year) format. "
    "Do not attempt to cite articles that are not in the context."
    "If you don't know the answer, say that you don't know."
    "Use up to five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)
"""
# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. "
    "What interventions might work best for me?"
)

response = llm.invoke(prompt)

print(response.content)




Based on the provided texts, here's what interventions might work best for you in Tanzania, given your lion/cattle conflict:

**1. Fortified Bomas ("Living Walls"):**

*   **Highly Effective:** The first study specifically conducted in Tanzania (near Tarangire National Park) found that fortified bomas – called "Living Walls" – were **99.9% effective** at preventing nighttime carnivore attacks over a 10-year period. Crucially, **no lion deaths occurred at fortified sites.**
*   **Recommendation:** This is the *most* strongly supported intervention for your situation, given the research was done in a similar context to yours.

**2. LED Flashlight Systems:**

*   **Effective Deterrent:** The second study (from Kenya, but relevant given the shared lion populations and challenges) showed that installing LED flashlight systems at bomas **significantly reduced attacks.**
*   **Shifts Behavior:**  Importantly, flashlights didn't just reduce attacks overall, they also caused lions to shift thei

In [88]:
documents = [
    Document(
        page_content=(
            "Elephants (Loxodonta africana) raiding crops around Tanzanian national parks threaten farmers’ lives and livelihoods, thus contributing to negative local attitudes towards wildlife. "
            "As a result, there is often tacit support for poaching among local communities, and elephants suffer through reprisal poisoning or wounding or through being shot as ‘problem animals’ by game wardens. "
            "Human-elephant conflict (HEC) is growing as the wildlands that still have elephants, especially around national parks, reserves, and wildlife corridors, are increasingly being settled. "
            "Sisal string fences soaked in engine oil mixed with ground chili (Capsicum spp.) can dissuade elephants from entering fenced fields. "
            "For the last nine years, farmers around Mikumi National Park in Tanzania have been constructing these fences around ripening crops, and there have been no incidents of fences being broken by elephants. "
            "Community-based organizations (CBOs) that manage members’ savings through village micro-credit associations help ensure the costs of materials and fence construction are met. "
            "Chili fences are rapidly becoming widespread, facilitated through farmer-to-farmer exchanges where teams of farmers demonstrate both the fences and the CBOs needed to support the project. "
            "We argue that promoting the use of chili fences, coupled with supporting CBOs, as a best practice within communities and government programs and budgets, will help reduce the need for HEC compensation, protect livelihoods, empower rural women, increase the food security of rural farmers, and help conserve elephants."
        ),
        metadata={
            "title": "Chili Fences and Community-Based Solutions to Human-Elephant Conflict in Tanzania",
        }
    ),
    Document(
        page_content=(
            "Community-Based Human-Elephant Conflict Mitigation: The value of an evidence-based approach in promoting the uptake of effective method\n"
            "DonnyGunaryadi a1111111111 a1111111111 a1111111111 a1111111111 a1111111111 1 , Sugiyo 1 , SimonHedges 2 *\n"
            "Humanelephant conflict (HEC) is a serious threat to elephants and can cause major eco nomic losses. It is widely accepted that reduction of HEC will often require community based methods for repelling elephants but there are few tests of such methods. "
            "We tested community-based crop-guarding methods with and without novel chili-based elephant deter rents and describe changes in farmers' willingness to adopt these methods following our demonstration of their relative effectiveness. "
            "In three separate field-trials that took place over almost two years (October 2005 May 2007) in two villages adjacent to Way Kambas National Park (WKNP) in Indonesia, we found that community-based crop-guarding was effective at keeping Asian elephants (Elephas maximus) out of crop fields in 91.2% (52 out of 57), 87.6% (156 out of 178), and 80.0% (16 out of 20) of attempted raids. "
            "Once the methodhadbeenshowntobeeffectiveat demonstration sites, farmers in 16 villages around WKNPvoluntarily adopted it during the July 2008 to March 2009 period and were able to repel elephants in 73.9% (150 out of 203) of attempted raids, with seven villages repelling 100%ofattempted raids. "
            "These 16 villages had all experienced high levels of HEC in the preceding years; e.g. they accounted for >97% of the 742 HEC incidents recorded for the entire park in 2006. "
            "Our work shows, therefore, that a simple evidence-based approach can facilitate significant reductions in HEC at the protected area scale."
        ),
        metadata={
            "title": "Community-Based Human-Elephant Conflict Mitigation in Indonesia",
        }
    ),
    Document(
        page_content=(
            "Beehive Fences as a MultiDimensional Conflict-Mitigation Tool for Farmers Coexisting with Elephants\n"
            "Lucy E. King,1,2∗ Fredrick Lala,3 Hesron Nzumu,4 Emmanuel Mwambingu,4 and Iain Douglas-Hamilton1,2\n"
            "Increasing habitat fragmentation and human population growth in Africa has resulted in an escalation in human–elephant conflict between small-scale farmers and free-ranging African elephants (Lox odonta Africana). "
            "In 2012 Kenya Wildlife Service (KWS) implemented the national 10-year Conservation and Management Strategy for the Elephant in Kenya, which includes an action aimed at testing whether beehive fences can be used to mitigate human–elephant conflict. "
            "From 2012 to 2015, we field-tested the efficacy of beehive fences to protect 10 0.4-ha farms next to Tsavo East National Park from elephants. We hung a series of beehives every 10 m around the boundary of each farm plot. The hives were linked with strong wire. "
            "After an initial pilot test with 2 farms, the remaining 8 of 10 beehive fences also contained 2-dimensional dummy hives between real beehives to help reduce the cost of the fence. Each trial plot had a neighboring control plot of the same size within the same farm. "
            "Of the 131 beehives deployed 88% were occupied at least once during the 3.5-year trial. Two hundred and fifty-three elephants, predominantly 20–45 years old entered the community farming area, typically during the crop- ripening season. "
            "Eighty percent of the elephants that approached the trial farms were kept out of the areas protected by the beehive fences, and elephants that broke a fence were in smaller than average groups. "
            "Beehive fences not only kept large groups of elephants from invading the farmland plots but the farmers also benefited socially and financially from the sale of 228 kg of elephant friendly honey. "
            "As news of the success of the trial spread, a further 12 farmers requested to join the project, bringing the number of beehive fence protected farms to 22 and beehives to 297. "
            "This demonstrates positive adoption of beehive fences as a community mitigation tool. Understanding the response of elephants to the beehive fences, the seasonality of crop raiding and fence breaking, and the willingness of the community to engage with the mitigation method will help contribute to future management strategies for this high human–elephant conflict hotspot and other similar areas in Kenya."
        ),
        metadata={
            "title": "Beehive Fences as Conflict-Mitigation Tools for Farmers and Elephants in Kenya",
        }
    )
]

# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: What should I do if elephants are destroying my crops? And what are the most cost-effective prevention methods, if there are any you know of?"
)

response = llm.invoke(prompt)

print(response.content)


#there were more texts I could have given but I wanted to see this first (also I have been giving primarily abstracts)

Okay, based on the provided texts, here's a breakdown of what you can do if elephants are destroying your crops, and the most cost-effective prevention methods:

**What to Do (and What *Not* to Do):**

*   **Avoid Retaliation:** The texts highlight that killing or harming elephants (even as "problem animals") often leads to a cycle of negative consequences – further conflict, potential poaching, and doesn't address the root of the problem.
*   **Report to Authorities:** While not explicitly stated as a first step, reporting incidents to game wardens or wildlife authorities is generally advisable, especially to document the problem and potentially access support. However, the texts emphasize *preventative* measures are key to reducing reliance on reactive responses from authorities.

**Cost-Effective Prevention Methods (Ranked by Apparent Effectiveness & Cost):**

1.  **Chili Fences:** (Most Promising & Relatively Low Cost)
    *   **How it Works:** Sisal string fences soaked in a mixtu

In [90]:
documents = [
    Document(
        page_content=(
            "Depredation of livestock by large carnivores is an important but poorly understood source of human–carnivore conflict. "
            "We examined patterns of livestock depredation by jaguars (Panthera onca) and pumas (Puma concolor) on a ranch–wildlife reserve in western Brazil to assess factors contributing to prey mortality. "
            "We predicted jaguars would kill a greater proportion of calves than yearling and adult cattle and that proximity to suitable habitat would increase mortality risk. "
            "We further speculated that exposure to predation risk would promote livestock grouping and increased movement distance. "
            "We recorded 169 cattle mortality incidents during 2003–2004, of which 19% were due to predation by jaguars and pumas. "
            "This level of mortality represented 0.2–0.3% of the total livestock holdings on the ranch. "
            "Jaguars caused most (69%) cattle predation events, and survival in allotments was lower for calves than for other age classes. "
            "Forest proximity was the only variable we found to explain patterns of livestock mortality, with predation risk increasing as distance to forest cover declined. "
            "Due to low predation risk, cattle movement patterns and grouping behavior did not vary relative to level of spatial overlap with radiocollared jaguars. "
            "The overall effect of predation on cattle was low and livestock likely constituted an alternative prey for large cats in our study area. "
            "However, selection of calves over other age cohorts and higher predation risk among cattle in proximity to forest cover is suggestive of selection of substandard individuals. "
            "Cattle ranchers in the Pantanal region may reduce cattle mortality rates by concentrating on losses due to nonpredation causes that could be more easily controlled."
        ),
        metadata={
            "title": "Evaluation of Potential Factors Predisposing Livestock to Predation by Jaguars"
        }
    ),
    Document(
        page_content=(
            "Most large carnivore species are in global decline. Conflicts with people, particularly over depredation on small and large livestock, is one of the major causes of this decline. "
            "Along tropical deforestation frontiers, large felids often shift from natural to livestock prey because of their increased proximity to human agriculture, thus increasing the likelihood of conflicts with humans. "
            "On the basis of data from 236 cattle ranches, we describe levels of depredation by jaguars (Panthera onca) and pumas (Puma concolor) on bovine herd stocks and examine the effects of both landscape structure and cattle management on the spatial patterns and levels of predation in a highly fragmented forest landscape of southern Brazilian Amazonia. "
            "Generalized linear models showed that landscape variables, including proportion of forest area remaining and distance to the nearest riparian forest corridor, were key positive and negative determinants of predation events, respectively. "
            "We detected clear peaks of depredation during the peak calving period at the end of the dry season. Bovine herd size and proportion of forest area had positive effects on predation rates in 60 cattle ranches investigated in more detail. "
            "On the other hand, distance from the nearest riparian forest corridor was negatively correlated with the number of cattle predated. "
            "The mean proportion of cattle lost to large felids in 24 months for the region varied according to the herd class size (≤500: 0.82%; 500–1500: 1.24%; ≥1500: 0.26%) but was never greater than 1.24%. "
            "The highest annual monetary costs were detected in large cattle ranches (≥1500 head of cattle), reaching US$ 885.40. "
            "Patterns of depredation can be explained by a combination of landscape and livestock management variables such as proportion of forest area, distance to the nearest riparian corridor, annual calving peak, and bovine herd size."
        ),
        metadata={
            "title": "Human-Wildlife Conflicts in a Fragmented Amazonian Forest Landscape: Determinants of Large Felid Depredation on Livestock"
        }
    )
]
# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: I know jaguars can prey on goats and cattle, which I have; what measures can I take to save them from getting harmed? Can you check these pdfs to see which ones might help?"
)

response = llm.invoke(prompt)

print(response.content)

Okay, based on the two provided texts, here's a breakdown of measures you can take to protect your goats and cattle from jaguar and puma predation, along with how the texts support those recommendations.  I'll prioritize based on what the studies *specifically* found to be effective or related to risk.

**Key Takeaways & Recommended Measures (Prioritized):**

1. **Manage Proximity to Forest Cover/Riparian Corridors:** This is the *most* consistently highlighted factor in both studies.
   * **What to do:**  If possible, avoid grazing livestock in areas immediately adjacent to forests or riparian (river/stream-side) corridors. The closer the cattle are to these areas, the higher the predation risk.  The second study specifically notes distance to riparian corridors as *negatively* correlated with predation – meaning further away is safer.
   * **Why it works:**  These areas provide cover and travel routes for jaguars and pumas. They are hunting grounds.
   * **Evidence:** Both studies em

In [92]:
documents = [
    Document(
        page_content=(
            "Estimates indicate coyotes (Canis latrans) kill an average of 1–2.5% of the domestic adult sheep and 4–9% of the lambs "
            "in the 17 western states. Livestock producers reduce losses (mortality) by using various livestock management practices, "
            "frightening devices, trapping, snaring, calling and shooting, sodium cyanide guns, denning, aerial gunning, and livestock guarding dogs. "
            "Livestock producers in the U.S. began using guarding dogs to protect domestic sheep from predators during the mid-1970s. "
            "Their effectiveness has been evaluated in enclosures and under ranching conditions. These evaluations usually compared losses before and after producers obtained guarding dogs. "
            "In this study, the effectiveness of livestock guarding dogs for protecting domestic sheep from predators was determined from two postal and three telephone surveys. "
            "A general survey was mailed to all 433 Colorado Wool Grower Association members. The survey requested information on type of operation (fenced pasture, open range, feedlot); "
            "the number of ewes and lambs in an operation; ewe and lamb mortality from all causes (weather, disease, predation, etc.); and ewe and lamb mortality from coyotes, mountain lions (Felis concolor), "
            "black bears (Ursus americanus), and other predators. Responses from 124 of 174 producers that completed the general mail survey were used to estimate sheep mortalities for producers without guarding dogs. "
            "A random sample of 29 nonrespondents was surveyed by telephone to determine if there was a nonresponse bias. Responses from 21 of these producers were used to estimate herd sizes and mortality rates for nonrespondents without guarding dogs. "
            "Respondents with guarding dogs, respondents without sheep, those with sheep primarily in feedlots, and incomplete responses were eliminated from both surveys. "
            "This study compares domestic sheep losses in fenced pastures and on open ranges both with and without livestock guarding dogs in Colorado during 1986 and provides producer estimates of the value of sheep saved from predators by guarding dogs."
        ),
        metadata={
            "title": "Effective Use of Livestock Guarding Dogs for Reducing Predation on Domestic Sheep"
        }
    ),
    Document(
        page_content=(
            "Coyotes (Canis latrans) are showing increasingly bold behaviors toward people and their pets throughout North America. "
            "Bold behavior by wildlife might be reduced by hazing and aversive conditioning, which is recommended in many management plans for coyotes, "
            "but there is little information about how it is to be conducted, and few studies have tested this approach. "
            "We conducted an online search for coyote management plans across North America to review techniques and recommendations related to hazing or aversive conditioning, "
            "and reported on the implementation of a high-intensity aversive conditioning program in Calgary, Alberta, Canada. "
            "Almost all the management plans recommended hazing coyotes, most often by using a combination of noises, lights, and movements, "
            "but only 20% of 71 plans recommended high-intensity techniques like those used by the contractors in Calgary. "
            "Contractors there searched for coyotes in 72 public park areas where members of the public had submitted reports to a civic call center for bold coyotes. "
            "They attended sites on 1,917 occasions, observed coyotes on 765 occasions, and reported coyote treatments and responses on 734 occasions. "
            "The probability of coyote retreats increased by 29%–37% with each additional previous aversive conditioning event at the site, and doubled with the presence of dogs and with the application of projected chalk balls. "
            "During engagements with contractors, coyote retreat probability declined by 21%–25% with each additional day since the last aversive conditioning event, "
            "and by 97.2%–97.6% with the presence of dogs and when shots were fired from a paintball gun, presumably because these tools were used only on the boldest coyotes. "
            "We found no effect of the presence or past number of aversive conditioning events on the number of coyote reports per week by the public. "
            "Although such high-intensity aversive conditioning is rarely recommended in management plans, our results suggest that its repeated application can reduce coyote boldness over time, "
            "though coyotes may not generalize this response to other people owing to visual cues (e.g., high-visibility vests, consistent vehicles) associated with contractors."
        ),
        metadata={
            "title": "Proactive Use of Intensive Aversive Conditioning Increases Probability of Retreat by Coyotes"
        }
    )
]
# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best and incentivize them to stay away?"
)

response = llm.invoke(prompt)

print(response.content)

Okay, let's break down what these studies tell us about large carnivore (and by extension, potentially coyote) predation on cattle, and how that might inform your strategy. While these studies focus on jaguars and pumas in Brazil, the underlying principles can be applied to coyote behavior.

**Key Takeaways from the Studies (and how they relate to coyotes):**

*   **Calves are Vulnerable:** Both studies consistently show that calves are disproportionately targeted. This is likely due to their smaller size, lower defensive capabilities, and potentially their higher nutritional value. *This is a critical point for your situation.*
*   **Habitat Proximity Matters:** Predation risk increases closer to cover (forest in the studies, but this translates to brush, woods, or any sheltered area for coyotes). Coyotes will use these areas for ambush points and escape routes.
*   **Herd Size & Density:** Larger herds *can* experience higher overall predation numbers (more targets), but smaller herd

In [96]:
documents += [
    Document(
        page_content=(
            "Population expansion of deer and the damage to planted trees caused by them is becoming a serious conflict between deer and humans. Although there are many methods for tree protection, their effectiveness varies from case to case, and the determinant of effectiveness has rarely been clarified. "
            "We investigated the following three specific questions: 1) What factors influence forest managers’ choice of protection methods? 2) What factors were important in reducing tree damage by each protection method? 3) Which of the repellents and fences was more effective in reducing tree damage? "
            "In 2018, we obtained data of plantation tree damages caused by sika deer (Cervus nippon) and tree protection methods from 1424 plantations in Japan. No protection was chosen at low deer densities, and repellents and fences were chosen at middle and high densities by forest managers. "
            "In the case of repellents, the damage rank increased with increasing deer density, tree age, and maximum snow depth, but was not affected by planted tree species (Cryptomeria japonica and Chamaecyparis obtusa). "
            "In the case of fences, the damage rank increased with increasing deer density, tree age, maximum snow depth, slope, and decreasing net height. The damage rank also increased with the usage of separable digging prevention net and the net without stainless wire. "
            "Deer protection methods should be selected according to environmental conditions such as deer density, snow depth, and slope degree in the case of the treated tree species."
        ),
        metadata={
            "title": "Fences are More Effective than Repellents in Reducing Deer Browsing on Planted Two Conifer Species"
        }
    ),
    Document(
        page_content=(
            "William L. Palmer, Jack M. Payne, Robert G. Wingard, John L. George\n"
            "White-tailed deer (Odocoileus virginianus) have fed on agricultural crops since settlers first planted fields on homesteads. In colonial days, deer populations were relatively sparse, farms were small, families were large, and fields were attended regularly. Damage by deer was controlled by hunting, and costs were offset by substituting venison for lost crops. "
            "Today, deer herd numbers are large, agricultural fields are larger and attended infrequently, and deer damage to crops is both widespread and locally severe. Farmers have used repellents, fencing, and firearms to discourage deer from damaging crops; repellents and fencing generally have been ineffective or too costly. "
            "Hunting, although effective in reducing deer numbers, is not possible in many areas because of land-use conflicts. Research at The Pennsylvania State University was conducted to develop a low-cost alternative to the traditional 2.4-m woven wire barrier fence, considered to be deer-proof but too expensive for most agricultural uses. "
            "This study evaluates the effectiveness of alternative fence designs in reducing deer damage and quantifies the cost-benefit relationships of fencing on various crops."
        ),
        metadata={
            "title": "A Practical Fence to Reduce Deer Damage"
        }
    ),
    Document(
        page_content=(
            "White-tailed deer (Odocoileus virginianus) may cause more damage than any other species of wildlife. These damages include crop loss, automobile and aviation collisions, disease transmission, environmental degradation, and destruction of ornamental plantings. "
            "One practical method of controlling deer damage is the use of exclusionary fences. The relatively high cost of labor and materials required to build effective fences has limited most applications to the protection of orchards, vegetable farms, other high-value resources, and mitigation of human health and safety risks. "
            "Improvements in fence technology resulting in less expensive, yet effective fences have expanded the use of fences to manage damage caused by deer. Fences typically installed to manage white-tailed deer damage include wire or plastic mesh, electrified high-tensile steel wire, and electrified polytape or polyrope fence. "
            "This review of scientific literature on fencing determines which fence designs would be the most effective for excluding deer in a variety of situations."
        ),
        metadata={
            "title": "Fences and Deer Damage Management: A Review of Designs and Efficacy"
        }
    )
]
# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: Deers keep destroying and taking over our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank?"
)

response = llm.invoke(prompt)

print(response.content)

Based on the provided texts, here's a breakdown of options for preventing deer damage to your agricultural fields, keeping cost in mind:

**1. Hunting (Potentially Cost-Effective, but with Limitations):**

*   Historically, hunting was a primary control method. However, the texts note that **hunting is often not possible today due to land-use conflicts.** If feasible in your area, it could be a relatively low-cost option.
*   In the past, venison from hunted deer offset crop losses, making it economically viable.

**2. Fencing (Variable Cost, Alternatives to Expensive Options):**

*   **Traditional woven wire fences (2.4m high)** are considered "deer-proof" but are **too expensive for most agricultural uses.**
*   **Research is focused on developing *low-cost* alternatives to woven wire.** The texts mention studies evaluating the effectiveness and cost-benefit of **alternative fence designs.**
*   **Electrified fences** (high-tensile steel wire, polytape, or polyrope) are mentioned as 

In [98]:
documents += [
    Document(
        page_content=(
            "Human-bear conflicts cause annoyance, financial losses, injuries, and even death to people. In poorer parts of the world, conflicts with bears can affect local economies. Retaliation against bears may threaten the future of small, isolated populations. "
            "Our survey of the world’s bear experts revealed that the problem is worsening in terms of severity of conflicts and their impact on bear conservation on all four continents inhabited by bears. However, the main drivers of conflict, and its manifestations, differ among bear species. "
            "We reviewed human-bear conflict management plans from which we identified 10 categories of mitigating interventions that together comprise a ubiquitous bear conflict management toolbox. Within this toolbox, the peer-reviewed literature indicates heavy reliance on education and physical barriers for conflict mitigation. "
            "In customizing these general approaches to local circumstances, it is important to be mindful of starkly varying geopolitical and social circumstances. There is a pressing need to improve transfer of knowledge from places with active empirical research on mitigation (especially North America), and adapting methodologies to other parts of the world. "
            "We saw little evidence of evaluation and adaptive management in the conflict plans. Failure to mitigate conflicts may reduce society’s tolerance of bears and diminish conservation efforts."
        ),
        metadata={
            "title": "Resolving Human Bear Conflict: A Global Survey of Countries, Experts, and Key Factors"
        }
    ),
    Document(
        page_content=(
            "Conflict between humans and black bears (Ursus americanus) occurs throughout North America with increasing public demand to replace lethal management with non-lethal methods, such as aversive conditioning (AC). "
            "AC aims to teach animals to associate negative stimuli with humans or their infrastructure. We sought to test the efficacy of AC using radio-collared black bears in Whistler, British Columbia, by monitoring individuals and assigning those in conflict with people to control or treatment groups. "
            "We measured wariness using overt reaction distance, displacement distance, and reaction to researchers before, during and after executing 3–5-day AC programs that consisted of launching projectiles at bears in the treatment group. "
            "We also assessed predictors of successful AC events (i.e., leaving at a run), changes in bear use of human-dominated habitat during the day and at night, and the effects of including a sound stimulus to signal the beginning and end of AC events. "
            "Among treated bears, overt reaction distance increased by 46.5% and displacement distance increased by 69.0% following AC programs, whereas both overt reaction distance and displacement distance decreased over time among control group bears. "
            "Each additional AC event during the previous 30 days increased likelihood of bear departure in response to researcher presence by 4.5%. The success of AC events varied among individuals, declined with distance to cover, and increased with exposure to previous AC events. "
            "Projectiles launched from guns were slightly more effective at causing bears to displace compared to those launched from slingshots, and sound stimuli decreased the likelihood of a successful AC event. "
            "AC did not alter diurnal use by bears of human-dominated habitat. Our results suggest that AC effectively increases short-term wariness in black bears but does not alter bear use of human-dominated spaces, highlighting the importance of proactive attractant management and prevention of food conditioning."
        ),
        metadata={
            "title": "Aversive Conditioning Increases short-term Wariness but does not change habitat use in black bears associated with conflict"
        }
    ),
    Document(
        page_content=(
            "Human-bear conflicts triggered by nuisance behaviour in public places and damage to livestock, crops, beehives and trees are among the main threats to bear populations globally. "
            "The effectiveness of interventions used to minimize bear-caused damage is insufficiently known and comparative reviews are lacking. "
            "We conducted a meta-analysis of 77 cases from 48 publications and used the relative risk of damage to compare the effectiveness of non-invasive interventions, invasive management (translocations) and lethal control (shooting) against bears. "
            "We show that the most effective interventions are electric fences (95% confidence interval = 79.2–100% reduction in damage), calving control (100%) and livestock replacement (99.8%), but the latter two approaches were applied in only one case each and need more testing. "
            "Deterrents varied widely in their effectiveness (13.7–79.5%) and we recommend applying these during the peak periods of damage infliction. "
            "We found shooting (−34.2 to 100%) to have a short-term positive effect with its effectiveness decreasing significantly and linearly over time. "
            "We did not find relationships between bear density and intervention effectiveness, possibly due to differences in spatial scales at which they were measured (large scales for densities and local fine scales for effectiveness). "
            "We appeal for more effectiveness studies and their scientific publishing in regard to under-represented conflict species and regions."
        ),
        metadata={
            "title": "Variation and Conservation Implications of the Effectiveness of Anti-Bear Interventions"
        }
    )
]

# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: We live in a suburb and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community without removing our fruit trees? Can you check these pdfs to see which ones might help?"
)

response = llm.invoke(prompt)

print(response.content)


Okay, based on the provided texts, here's a breakdown of the best ways to prevent bear-human conflict in a suburban setting like yours, focusing on keeping your fruit trees and minimizing attractants, drawing directly from the research.  I'll also indicate which documents support each recommendation.

**Key Strategies for Your Community (and Individual Actions):**

1. **Attractant Management is Crucial:** This is the *most* consistently emphasized point across all the documents relating to bear conflict.  Bears are drawn in by food sources.  Removing or securing these is the primary defense.
    *   **Secure Trash:** This is repeatedly mentioned as a major attractant.  Ensure all trash cans are bear-resistant or stored in a secure location (garage, shed) until collection day. ([Human-bear conflicts cause annoyance...](https://www.researchgate.net/publication/344099999_Human-bear_conflicts_cause_annoyance_financial_losses_injuries_and_even_death_to_people))
    *   **Manage Fruit Trees:

In [ ]:
documents += [
    Document(
        page_content=(
            "Mitigating conflicts associated with predation on livestock is essential for conserving large carnivores in human dominated landscapes. "
            "This is generally addressed by targeting individual management practices affecting predation risk, often disregarding that different livestock husbandry systems (i.e., groups of farms sharing similar resource bases, production patterns and management practices) with different vulnerabilities to predation may coexist within predator ranges, each of which requiring tailored prescriptions to reduce predation. "
            "Here we evaluated the importance of considering both husbandry systems and individual management practices to mitigate conflicts due to cattle predation by wolves in Portugal, where attacks on cattle increased N3 times in 1999–2013. "
            "Government records from 2012 to 2013 indicated that only b2% of cattle farms suffered wolf attacks, of which b4% had N10 attacks per year. "
            "We found that attacks were concentrated in the free-ranging husbandry system, which was characterized by multi-owner herds, largely grazing communal land far from shelter, and seldom confined. "
            "Protecting these herds at night in winter was the most important factor reducing wolf attacks, which could be achieved by changing practices of ≈25% of farmers in this system. "
            "Attacks were much lower in the semi-confined system, probably because herds grazed pastures closer to shelter, and they were often confined with fences or in barns. "
            "Farms bringing calves b3 months old to pastures were associated with about 90% of attacks, but changing this practice would involve ≈50% of farmers in this system. "
            "Our results underline the importance of identifying livestock husbandry systems and to adjust mitigation strategies to each system."
        ),
        metadata={
            "title": "Wolf Predation on Cattle in Portugal: Assessing the Effects of Husbandry Systems"
        }
    ),
    Document(
        page_content=(
            "Worldwide, native predators are killed to protect livestock, an action that can undermine wildlife conservation efforts and create conflicts among stakeholders. "
            "An ongoing example is occurring in the western United States, where wolves (Canis lupus) were eradicated by the 1930s but are again present in parts of their historic range. "
            "While livestock losses to wolves represent a small fraction of overall livestock mortality, the response to these depredations has resulted in widespread conflicts including significant efforts at lethal wolf control to reduce impacts on livestock producers, especially those with large-scale grazing operations on public lands. "
            "A variety of nonlethal methods have proven effective in reducing livestock losses to wolves in small-scale operations but in large-scale, open-range grazing operations, nonlethal management strategies are often presumed ineffective or infeasible. "
            "To demonstrate that nonlethal techniques can be effective at large scales, we report a 7-year case study where we strategically applied nonlethal predator deterrents and animal husbandry techniques on an adaptive basis (i.e., based on terrain, proximity to den or rendezvous sites, avoiding overexposure to techniques such as certain lights or sound devices that could result in wolves losing their fear of that device, etc.) to protect sheep (Ovis aries) and wolves on public grazing lands in Idaho. "
            "We collected data on sheep depredation mortalities in the protected demonstration study area and compared these data to an adjacent wolf-occupied area where sheep were grazed without the added nonlethal protection measures. "
            "Over the 7-year period, sheep depredation losses to wolves were 3.5 times higher in the Nonprotected Area (NPA) than in the Protected Area (PA). "
            "Furthermore, no wolves were lethally controlled within the PA and sheep depredation losses to wolves were just 0.02% of the total number of sheep present, the lowest loss rate among sheep-grazing areas in wolf range statewide, whereas wolves were lethally controlled in the NPA. "
            "Our demonstration project provides evidence that proactive use of a variety of nonlethal techniques applied conditionally can help reduce depredation on large open-range operations."
        ),
        metadata={
            "title": "Adaptive Use of Nonlethal Strategies for Minimizing Wolf-Sheep Conflict in Idaho"
        }
    ),
    Document(
        page_content=(
            "There is growing interest in developing effective interventions to manage socially and environmentally damaging conservation conflicts. "
            "There are a variety of intervention strategies that can be applied in various contexts, but the reasons one type of intervention is chosen over another remain underexplored. "
            "We surveyed conservation researchers and practitioners (n = 427) to explore how characteristics of conflicts and characteristics of decision makers influence recommendations to alleviate conservation conflict. "
            "Using a full-factorial design, we experimentally manipulated 3 aspects of the descriptions of 8 different wildlife-conflict scenarios (development status of the conflict country, conflict framing, and legality of killing wild animals) and recorded which of 5 intervention types (wildlife impact reduction, awareness, enforcement, economic incentives, or stakeholder engagement) respondents prioritized. "
            "We also recorded information on respondents’ demographic and disciplinary backgrounds. "
            "Stakeholder-based interventions were recommended most often in the survey and in written feedback. However, when we fitted multinomial mixed logit models with fully completed scenario responses (n = 411), recommendations were influenced by small changes in the details of conflict and differed according to respondent characteristics. "
            "Enforcement and awareness interventions were prioritized relatively more for conflicts in more highly developed nations and by respondents with more natural science backgrounds and relatively less experience with conflict. "
            "Contrastingly, economic interventions were prioritized more when wildlife killing was described as illegal. "
            "Age, gender, and development status of the respondent’s home country also predicted some intervention decisions. "
            "Further, interrogating the influences shaping conservation decision making will further help in the development of evidence-informed interventions."
        ),
        metadata={
            "title": "Predicting Intervention Priorities for Wildlife Conflict"
        }
    )
]

# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: We live in a suburb and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community without removing our fruit trees? Can you check these pdfs to see which ones might help?"
)

response = llm.invoke(prompt)

print(response.content)


In [100]:
documents = [
    Document(
        page_content=(
            "Mitigating conflicts associated with predation on livestock is essential for conserving large carnivores in human dominated landscapes. "
            "This is generally addressed by targeting individual management practices affecting predation risk, often disregarding that different livestock husbandry systems (i.e., groups of farms sharing similar resource bases, production patterns and management practices) with different vulnerabilities to predation may coexist within predator ranges, each of which requiring tailored prescriptions to reduce predation. "
            "Here we evaluated the importance of considering both husbandry systems and individual management practices to mitigate conflicts due to cattle predation by wolves in Portugal, where attacks on cattle increased N3 times in 1999–2013. "
            "Government records from 2012 to 2013 indicated that only b2% of cattle farms suffered wolf attacks, of which b4% had N10 attacks per year. "
            "We found that attacks were concentrated in the free-ranging husbandry system, which was characterized by multi-owner herds, largely grazing communal land far from shelter, and seldom confined. "
            "Protecting these herds at night in winter was the most important factor reducing wolf attacks, which could be achieved by changing practices of ≈25% of farmers in this system. "
            "Attacks were much lower in the semi-confined system, probably because herds grazed pastures closer to shelter, and they were often confined with fences or in barns. "
            "Farms bringing calves b3 months old to pastures were associated with about 90% of attacks, but changing this practice would involve ≈50% of farmers in this system. "
            "Our results underline the importance of identifying livestock husbandry systems and to adjust mitigation strategies to each system."
        ),
        metadata={
            "title": "Wolf Predation on Cattle in Portugal: Assessing the Effects of Husbandry Systems"
        }
    ),
    Document(
        page_content=(
            "Worldwide, native predators are killed to protect livestock, an action that can undermine wildlife conservation efforts and create conflicts among stakeholders. "
            "An ongoing example is occurring in the western United States, where wolves (Canis lupus) were eradicated by the 1930s but are again present in parts of their historic range. "
            "While livestock losses to wolves represent a small fraction of overall livestock mortality, the response to these depredations has resulted in widespread conflicts including significant efforts at lethal wolf control to reduce impacts on livestock producers, especially those with large-scale grazing operations on public lands. "
            "A variety of nonlethal methods have proven effective in reducing livestock losses to wolves in small-scale operations but in large-scale, open-range grazing operations, nonlethal management strategies are often presumed ineffective or infeasible. "
            "To demonstrate that nonlethal techniques can be effective at large scales, we report a 7-year case study where we strategically applied nonlethal predator deterrents and animal husbandry techniques on an adaptive basis (i.e., based on terrain, proximity to den or rendezvous sites, avoiding overexposure to techniques such as certain lights or sound devices that could result in wolves losing their fear of that device, etc.) to protect sheep (Ovis aries) and wolves on public grazing lands in Idaho. "
            "We collected data on sheep depredation mortalities in the protected demonstration study area and compared these data to an adjacent wolf-occupied area where sheep were grazed without the added nonlethal protection measures. "
            "Over the 7-year period, sheep depredation losses to wolves were 3.5 times higher in the Nonprotected Area (NPA) than in the Protected Area (PA). "
            "Furthermore, no wolves were lethally controlled within the PA and sheep depredation losses to wolves were just 0.02% of the total number of sheep present, the lowest loss rate among sheep-grazing areas in wolf range statewide, whereas wolves were lethally controlled in the NPA. "
            "Our demonstration project provides evidence that proactive use of a variety of nonlethal techniques applied conditionally can help reduce depredation on large open-range operations."
        ),
        metadata={
            "title": "Adaptive Use of Nonlethal Strategies for Minimizing Wolf-Sheep Conflict in Idaho"
        }
    ),
    Document(
        page_content=(
            "There is growing interest in developing effective interventions to manage socially and environmentally damaging conservation conflicts. "
            "There are a variety of intervention strategies that can be applied in various contexts, but the reasons one type of intervention is chosen over another remain underexplored. "
            "We surveyed conservation researchers and practitioners (n = 427) to explore how characteristics of conflicts and characteristics of decision makers influence recommendations to alleviate conservation conflict. "
            "Using a full-factorial design, we experimentally manipulated 3 aspects of the descriptions of 8 different wildlife-conflict scenarios (development status of the conflict country, conflict framing, and legality of killing wild animals) and recorded which of 5 intervention types (wildlife impact reduction, awareness, enforcement, economic incentives, or stakeholder engagement) respondents prioritized. "
            "We also recorded information on respondents’ demographic and disciplinary backgrounds. "
            "Stakeholder-based interventions were recommended most often in the survey and in written feedback. However, when we fitted multinomial mixed logit models with fully completed scenario responses (n = 411), recommendations were influenced by small changes in the details of conflict and differed according to respondent characteristics. "
            "Enforcement and awareness interventions were prioritized relatively more for conflicts in more highly developed nations and by respondents with more natural science backgrounds and relatively less experience with conflict. "
            "Contrastingly, economic interventions were prioritized more when wildlife killing was described as illegal. "
            "Age, gender, and development status of the respondent’s home country also predicted some intervention decisions. "
            "Further, interrogating the influences shaping conservation decision making will further help in the development of evidence-informed interventions."
        ),
        metadata={
            "title": "Predicting Intervention Priorities for Wildlife Conflict"
        }
    ),
    Document(
        page_content=(
            "Estimates indicate coyotes (Canis latrans) kill an average of 1-2.5% of the domestic adult sheep and 4-9% of the lambs in the 17 western states (U.S. Fish and Wildl. Serv. 1978, Pearson 1986). "
            "Livestock producers reduce losses (mortality) by using various livestock management practices, frightening devices, trapping, snaring, calling and shooting, sodium cyanide guns, denning, aerial gunning, and livestock guarding dogs (Andelt 1987). "
            "Livestock producers in the U.S. began using guarding dogs to protect domestic sheep from predators during the mid-1970's (McGrew and Blakesley 1982, Pfeifer and Goos 1982). "
            "Their effectiveness has been evaluated in enclosures (McGrew and Blakesley 1982) and under ranching conditions (Linhart et al. 1979; Pfeifer and Goos 1982; Coppinger et al. 1983; Green and Woodruff 1983, 1988, 1990; Green et al. 1984; Andelt 1985; Black and Green 1985). "
            "These evaluations usually compared losses before and after producers obtained guarding dogs. "
            "In this study, I compare domestic sheep losses in fenced pastures and on open ranges both with and without livestock guarding dogs in Colorado during 1986. "
            "I also provide producer estimates of the value of sheep saved from predators by guarding dogs. "
            "A general survey was mailed to all (433) Colorado Wool Grower Association members. The survey requested information on type of operation (fenced pasture, open range, feedlot); the number of ewes and lambs in an operation; ewe and lamb mortality from all causes (weather, disease, predation, etc.); and ewe and lamb mortality from coyotes, mountain lions (Felis concolor), black bears (Ursus americanus), and other predators. "
            "Responses from 124 of 174 producers that completed the general mail survey were used to estimate sheep mortalities for producers without guarding dogs. "
            "A random sample of 29 of the nonrespondents was surveyed by telephone to determine if there was a nonresponse bias. "
            "Responses from 21 of these producers were used to estimate herd sizes and mortality rates for nonrespondents without guarding dogs. "
            "Respondents with guarding dogs, respondents without sheep, respondents with sheep primarily in feedlots, and incomplete responses were eliminated from both surveys."
        ),
        metadata={
            "title": "Effective of Livestock Guarding Dogs for Reducing Predation on Domestic Sheep"
        }
    ),
    Document(
        page_content=(
            "There is no proverbial silver bullet for mitigating human-wildlife conflict, but the study of animal behaviour is foundational to solving issues of coexistence between people and wild animals. "
            "Our purpose is to examine the theoretical and applied role that behavioural principles play in understanding and mitigating human-wildlife conflict, and delineate gaps in behavioural theory relative to mitigating these conflicts. "
            "Specifically, we consider two different, yet contemporary, examples of human-wildlife conflict: animal-vehicle collisions and carnivore depredation of livestock. "
            "Although ostensibly unrelated, both conflict areas share common themes relative to animal behavioural responses to disturbance and perception of risk. "
            "We first place the effects on wildlife due to these conflicts in the scope of population sustainability, and then examine current research relative to the following three questions. "
            "How is behavioural ecology relevant to these particular areas of conflict? "
            "Are advances toward understanding the mechanisms by which animals process information and make decisions being translated into management methods? "
            "How might management efforts be affected over time by individual behaviours, method integration and habituation/sensitization? "
            "Regarding animal-vehicle collisions, only in the last decade have researchers applied an antipredator theoretical framework with sensory ecology to understand aspects of marine mammal, terrestrial mammal and bird responses to vehicle approach, speed and associated stimuli. "
            "However, the size and speeds of modern vehicles demand that we improve economic models and possibly develop novel theoretical frameworks to better predict animal responses to vehicle approach. "
            "Within the context of carnivore-livestock depredation, our understanding of individual predator behaviour relative to perceived risk and factors contributing to the development of problem individuals will influence the efficacy of the most promising, nonlethal management approaches (e.g. distractive techniques, reproductive inhibition and olfactory barriers). "
            "In both cases, successful management is contingent upon a mechanistic understanding of how animals respond to disturbance and the information utilized to assess risk."
        ),
        metadata={
            "title": "No Single Solution: Application of Behavioural Principles in Mitigating Wildlife-Human Conflict"
        }
    ),
    Document(
        page_content=(
            "Gray wolves (Canis lupus) are a top predator in northern Asian ecosystems and often perceived as a threat to livestock. "
            "As a result, wolves are heavily persecuted and populations have declined throughout much of the region. "
            "Understanding the dynamics of wolf-livestock conflict is important for developing conservation actions that benefit wolves and human livelihoods. "
            "We measured the influence of landscape factors on patterns of wolf-livestock conflict in Ikh Nart Nature Reserve, Mongolia by modeling livestock predation risk using a partitioned Mahalanobis D2 (k) analysis. "
            "We based the model on 44 known predation sites obtained through 102 interviews with rural pastoralists and mapped risk at a 500 m spatial scale. "
            "Four factors strongly influenced predation risk at a given site in the landscape including distance to nearest ger camp, and the percent of surrounding tall vegetation, shrubland, and forbland habitat. "
            "Our results indicate that wolves tend to kill livestock in areas where their detection by humans and livestock is low. "
            "Managing wolves in Mongolia will require reducing livestock predation and subsequent retribution killing. "
            "This may be achieved by focusing conservation in areas where predation risk is highest, such as habitats with greater vegetation cover and areas near particular ger sites."
        ),
        metadata={
            "title": "Measuring and Mapping the Influence of Landscape Factors on Livestock Predation by Wolves in Mongolia"
        }
    ),
    Document(
        page_content=(
            "Environmental education is a widespread, yet relatively unexamined strategy to reduce human-wildlife conflicts. "
            "We evaluated knowledge, attitudes and behavioral intentions toward bear conservation after five years of environmental education in a Quichua community. "
            "Conflicts with livestock predation created mixed attitudes and behaviors toward bear conservation. "
            "Some program objectives were achieved, such as 88% of participants reported satisfaction with environmental knowledge gained. "
            "Behavioral intentions to decrease bear conflicts increased, and multiple regression analysis revealed support for the project was associated with program participation. "
            "Focus group meetings with teachers, local policy makers and para-biologists provided a context for recommendations to improve program success and revealed new issues for better bear management."
        ),
        metadata={
            "title": "Human-Wildlife Conflict and Environmental Education: Evaluating a Community Program to Protect the Andean Bear in Ecuador"
        }
    )
]

# Combine all document texts into one string as context
context = "\n\n".join(doc.page_content for doc in documents)

# Build the prompt with the question and context
prompt = (
    f"Context:\n{context}\n\n"
    "Question: If we live in an area with a lot of wolves, what cattle husbandry strategies should I employ to prevent any sort of wildlife-human conflict? Can you check these pdfs to see which ones might help?"
)

response = llm.invoke(prompt)

print(response.content)


Okay, based on the provided PDFs, here's a breakdown of cattle husbandry strategies to prevent wolf-livestock conflict, tailored to different situations, and drawing from the research presented.  I'll categorize them by the "husbandry system" approach highlighted in the first paper, and then add some general best practices.

**Key Takeaway:** The most effective strategies *depend on your specific situation* – whether you have a free-ranging or semi-confined system. A one-size-fits-all approach won't work.



**1. Free-Ranging Husbandry System (Herds grazing communal land, far from shelter, seldom confined - Portugal Study)**

This system is the *most vulnerable* according to the Portugal study.  Here's what to do:

*   **Night Protection in Winter:** This is the *most important* factor identified in the Portugal study.  Confine cattle at night during the winter months.  This is achievable for about 25% of farmers in this system.
*   **Shelter Access:**  While the study notes herds are 